## Medical Logits Extraction
MIMIC is my missing dataset. I don't have the possibity to test the models(**mimic_nb** and **mimic_ch**) on the training data and on the testing data.

What is required is four forward passes for each dataset:
- on the training dataset
    - pre_sigmoid values (logits) and post_sigmoid values(predictions)
- on the test dataset 
    - pre_sigmoid values (logits) and post_sigmoid values(predictions)

Most of the code comes from your library

In [7]:
# Constant to set 
dataset_dir = None
batch_size = None

# Every combination has to be setted 00,01,10,11
nb = False # It has to be changed manually
sigmoid = False # It has to be changed manually
    

In [3]:
import torch
import torchvision.transforms as trn
import torchxrayvision as xrv
import numpy as np

xray_transform = trn.Compose([  xrv.datasets.XRayCenterCrop(), xrv.datasets.XRayResizer(224)])

### Datasets and models for the forward pass

In [ ]:
if nb == True:
    dataset = xrv.datasets.MIMIC_Dataset(
                                imgpath = dataset_dir + "/MIMIC/images-224/files",
                                csvpath = dataset_dir + "/MIMICCXR-2.0/mimic-cxr-2.0.0-negbio.csv.gz",
                                metacsvpath = dataset_dir + "/MIMICCXR-2.0/mimic-cxr-2.0.0-metadata.csv.gz",
                                transform=xray_transform, data_aug=None)
    model = xrv.models.DenseNet(weights="mimic_nb").cuda()
    
else:
    dataset = xrv.datasets.MIMIC_Dataset(
                                imgpath = dataset_dir + "/images-224-MIMIC/files",
                                csvpath = dataset_dir + "/MIMICCXR-2.0/mimic-cxr-2.0.0-chexpert.csv.gz",
                                metacsvpath = dataset_dir + "/MIMICCXR-2.0/mimic-cxr-2.0.0-metadata.csv.gz",
                                transform=xray_transform, data_aug=None, unique_patients=False)
    model = xrv.models.DenseNet(weights="mimic_ch").cuda() 
    
xrv.datasets.relabel_dataset(xrv.datasets.default_pathologies , dataset) 

if sigmoid == False:
    # Remove sigmoids and normalization based on threshold
    model.apply_sigmoid = False
    model.op_threshs = None # It removes sigmoids and also """ the normalization of the outputs according to operating points for a given model""
    

#### Use the same split to retrive the Training Set and the Test Set

In [ ]:
gss = sklearn.model_selection.GroupShuffleSplit(train_size=0.8,test_size=0.2, random_state=0) #zero seed is used in torchxrayvision
train_inds, test_inds = next(gss.split(X=range(len(dataset)), groups=dataset.csv.patientid))
train_dataset = xrv.datasets.SubsetDataset(dataset, train_inds)
test_dataset = xrv.datasets.SubsetDataset(dataset, test_inds)


In [ ]:
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                            shuffle=False, num_workers=4, pin_memory=True) 

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                           shuffle=False, num_workers=4, pin_memory=True)

## Forward Pass

In [5]:
def get_prediction(loader):
    device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' ) 
    _output = []

    with torch.no_grad():
        for batch_idx, (data_tmp) in enumerate(loader):

            data=data_tmp['img'].to(device)

            output = model(data)

            _output.append(to_np(output))

    concat_output = concat(_output).copy()
    num_features = concat_output.shape[1]
    colnames = [f'_{i}' for i in range(num_features)]
    return  pd.DataFrame(concat_output, columns=colnames)


## Test and Store

In [ ]:
data_frame_train = get_prediction(train_loader)
data_frame_train.to_csv(f'train_nb_{nb}_sigmoid_{sigmoid}',index=False)

data_frame_test = get_prediction(test_loader)
data_frame_test.to_csv(f'test_nb_{nb}_sigmoid_{sigmoid}',index=False)

In [8]:
print(f'train_nb_{nb}_sigmoid_{sigmoid}')

train_nb_False_sigmoid_False
